<a href="https://colab.research.google.com/github/neoyipeng2018/acct725_teachingmaterials/blob/main/LLM_Lesson_2_PE_(EMPTY).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install groq --quiet
!pip show groq #0.23.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.6/129.6 kB 10.9 MB/s eta 0:00:00
Name: groq
Version: 0.26.0
Summary: The official Python library for the groq API
Home-page: https://github.com/groq/groq-python
Author: 
Author-email: Groq <support@groq.com>
License: Apache-2.0
Location: /usr/local/lib/python3.11/dist-packages
Requires: anyio, distro, httpx, pydantic, sniffio, typing-extensions
Required-by: 


In [ ]:
import os
import getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("GROQ_API_KEY")

# Prompt Engineering Tips

# Baseline: `Classify the sentiment` on 20 hard sentiment data points for finance

In [ ]:
!pip install datasets -qU

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 17.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-nvrtc-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-runtime-cu12==12.4.127; platform_system == "Linux" and platform_

In [ ]:
from datasets import load_dataset, Dataset, Value

# Load both splits
fpb_50agree = load_dataset("takala/financial_phrasebank", "sentences_50agree", trust_remote_code=True)["train"]
fpb_100agree = load_dataset("takala/financial_phrasebank", "sentences_allagree", trust_remote_code=True)["train"]

# Sets for fast difference calculation
set_50 = set(fpb_50agree['sentence'])
set_100 = set(fpb_100agree['sentence'])

# Sentences in 50agree not in 100agree
diff_sentences = set_50 - set_100

hardest_sentences = [
    "The corresponding share capital increase , EUR 1,012,945.50 was entered in the Trade Register today .",
    "The category was marked by maturity and the recession .",
    "The annual Thesis contest is aimed to enhance the thesis quality and to strengthen the thesis culture in polytechnics .",
    "A state program has been worked out , legal provisions have been developed , and rules and normative documents have been specified for the functioning of a technological park .",
    "This is due to the postponement of the decision-making of projects in the negotiation phase from the earlier estimation .",
    "The order consists of capacity expansion , maintenance services and new charging functionality , the company said .",
    "The bank also expects additional costs , related to the Dash 8 - Q400 jets , which the group shelved in October 2007 .",
    "The construction project is scheduled to start in the second quarter of 2009 and the new building is scheduled to be in place by the end of 2010 .",
    "The Finnish company previously said its operating result will be lower than the break-even posted a year earlier .",
    "Poyry is a global expert in consulting and engineering .",
    "Body The credit falls due February 24 , 2014 .",
    "The implementation of the deal is subject to the approval by the Finnish Competition Authority .",
    "The adjustment measures will be carried out in phases to secure the delivery capability .",
    "The right margin will be viewed separately in detail with every customer .",
    "After that the two companies sat down and sewed up the present deal .",
    "It also has potential clients in the growing environmental and recycling technology sectors .",
    "Public services will also be available .",
    "The world of glass is coming to Egypt and we invite Visitors from all around the world to attend Glass World Exhibition 2009 , Register Now at www.glassworldex.com",
    "The use case dramatically narrows if you go only with the hot s Ltd.",
    "Estimations indicate that even up to 170 different tablet computers or reading devices will be available in 2011 ."
]

# Filter original 50agree for rows with those sentences
filtered_rows = [row for row in fpb_50agree if row['sentence'] in hardest_sentences]

# Create new HuggingFace Dataset
diff_dataset = Dataset.from_list(filtered_rows)
diff_dataset   = diff_dataset.cast_column("label", Value("string"))

print(f"Total unique sentences: {len(diff_dataset)}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/8.88k [00:00<?, ?B/s]

financial_phrasebank.py:   0%|          | 0.00/6.04k [00:00<?, ?B/s]

FinancialPhraseBank-v1.0.zip:   0%|          | 0.00/682k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/4846 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/2264 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/20 [00:00<?, ? examples/s]

Total unique sentences: 20


In [ ]:
sentiments = {
    '0': "Negative",
    '1': "Neutral",
    '2': "Positive"
}

diff_dataset = diff_dataset.map(lambda ex: {
    "label":  sentiments[ex["label"]],
})

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

In [ ]:
!pip install transformers --quiet
!pip show transformers #4.51.3

Name: transformers
Version: 4.52.3
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.11/dist-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: peft, sentence-transformers


In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-classification", model="yiyanghkust/finbert-tone", device=0, truncation=True, max_length=512)
# pipe = pipeline("text-classification", model="ProsusAI/finbert", device=0, truncation=True, max_length=512)

config.json:   0%|          | 0.00/533 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/439M [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
results = pipe(hardest_sentences)
res = [x == y for x, y in zip(diff_dataset['label'], [r['label'].title() for r in results])]
accuracy = sum(res) / len(res)
print(f"Accuracy on hard sentences for FinBERT: {accuracy:.4%} ({sum(res)}/{len(res)})")

Accuracy on hard sentences for FinBERT: 70.0000% (14/20)


## Baseline with llama8k

In [ ]:
from groq import Groq

client = Groq()

In [ ]:
def sent_groq(txt):
  completion = client.chat.completions.create(
      model="llama3-8b-8192",
      messages=[
          {
              "role": "system",
              "content": (
                  "You are a sentiment classifier. "
                  "When asked for sentiment, output only one of: "
                  "`Positive`, `Neutral`, or `Negative`."
              ),
          },
          {
              "role": "user",
              "content": (
                  f"What is the sentiment of the news headline?\n\n"
                  f"{txt}\n\n"
                  "OUTPUT ONE OF THREE LABELS: 'Positive'/'Neutral'/'Negative'"
              ),
          },
      ],
      temperature=0,
      max_completion_tokens=1,
  )
  return completion.choices[0].message.content

In [ ]:
llama = [sent_groq(x) for x in hardest_sentences]

In [ ]:
res = [x == y for x, y in zip(diff_dataset['label'], llama)]
accuracy = sum(res) / len(res)
print(f"Accuracy on hard sentences baseline Llama 8B: {accuracy:.4%} ({sum(res)}/{len(res)})")

Accuracy on hard sentences baseline Llama 8B: 50.0000% (10/20)


## How can we improve it using basic prompt engineering techniques?
- use """ or ## to seperate instruction and context
- be more specific
- focus on what we want
- few shot


## Advanced: Use Chain of Thought to output the style and format